## 0. Importing Necessary Packages


In [1]:
import os                                 # Importing the os module which provides functions for interacting with the operating system
from moviepy.editor import (              # Importing functions from the MoviePy library, which is used for video editing.
                VideoFileClip,                # VideoFileClip allows you to load a video file and manipulate it as a clip object.
                concatenate_videoclips)       # concatenate_videoclips allows you to concatenate multiple video clips into one video.
import cv2                                # Importing cv2 module from OpenCV library, used for computer vision tasks (provides various functions to process images and videos).
import subprocess                         # Importing subprocess, commonly used for running external commands or scripts from within a Python script.
import tkinter                            # Importning tkinter, whch crates graphical user interfaces (GUIs) and provides a variety of widgets and controls to build desktop applications.
from tkinter import filedialog                # importing filedialog from tkinter, is a module in tkinter, which allows the user to select files or directories through a dialog window.
#ffmpegf

print("Everything was imported succesfully") #as terminal

Everything was imported succesfully


## 1. Defining the Relevant Directories, Variables & Functions

In [8]:
# ------------ DIRECTORIES -----------------------------
input_folder  = './videos_raw/'   # input folder with the raw video files (relative path) 
output_folder = './videos_split/' # output folder where the split videos will be saved (relative path) 

print("Input folder =", os.path.abspath(input_folder))
print("Output folder =", os.path.abspath(output_folder))


# ----------- VARIABLES --------------------------
video_extension = '.avi'  # Video format extension avi. Change as needed (e.g., .mp4)
codec_extensio  = 'XVID'  # Codec for avi. Consider other codecs for different extensions:
                            # '.mp4': 'libx264',
                            # '.avi': 'libxvid',
                            # '.mov': 'libx264',
                            # '.mkv': 'libx264',
                            # '.flv': 'flv',

num_cameras = 3          # Number of cameras for each "raw video". Change accordingly. 


# ----------- FUNCTIONS -------------------------
import cv2

def split_camera_views(input_file, output_files, num_cameras=3, codec_extension='mp4v'):
    """
    This split_camera_views function takes an input video file and splits it into three separate video files, 
    each corresponding to a different camera view. It assumes that the input video is composed 
    of three side-by-side camera views. The function outputs three video files, each containing 
    one of the camera views.

    Parameters:
    input_file (str): The path to the input video file.
    output_files (list of str): A list containing the paths to the output video files for each camera view.
    num_cameras (int): The number of camera views in the input video. Default is 3.
    codec_extension (str): The codec extension to be used for the output videos. Default is 'mp4v'.
    """
    
    cap = cv2.VideoCapture(input_file)  # Open the video file

    # Extract relevant information from the video
    width_per_camera = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // num_cameras  # Calculate width per camera
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Get the height of the video (same for all cameras)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))  # Get the frame rate of the video (same for all cameras)

    # Create VideoWriters for each camera
    fourcc = cv2.VideoWriter_fourcc(*codec_extension)  # Define the codec for the output videos
    out_cam1 = cv2.VideoWriter(output_files[0], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the first camera
    out_cam2 = cv2.VideoWriter(output_files[1], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the second camera
    out_cam3 = cv2.VideoWriter(output_files[2], fourcc, frame_rate, (width_per_camera, height))  # Create VideoWriter for the third camera

    while True:  # Loop to read frames from the video
        ret, frame = cap.read()  # Read a frame from the video

        # Check if the frame is None (end of video)
        if frame is None:  # If no frame is returned, break the loop
            break

        # Break the frame into three parts
        camera1_frame = frame[:, :width_per_camera, :]  # Extract the first camera view
        camera2_frame = frame[:, width_per_camera:2*width_per_camera, :]  # Extract the second camera view
        camera3_frame = frame[:, 2*width_per_camera:, :]  # Extract the third camera view

        # Display each camera view separately (optional)
        cv2.imshow('Camera 1', camera1_frame)  # Display the first camera view
        cv2.imshow('Camera 2', camera2_frame)  # Display the second camera view
        cv2.imshow('Camera 3', camera3_frame)  # Display the third camera view

        # Write frames to video files
        out_cam1.write(camera1_frame)  # Write the first camera view to its file
        out_cam2.write(camera2_frame)  # Write the second camera view to its file
        out_cam3.write(camera3_frame)  # Write the third camera view to its file

        if cv2.waitKey(1) == 27:  # Break the loop if 'Esc' key is pressed
            break

    # Release VideoWriters and VideoCapture
    out_cam1.release()  # Release the VideoWriter for the first camera
    out_cam2.release()  # Release the VideoWriter for the second camera
    out_cam3.release()  # Release the VideoWriter for the third camera
    cap.release()  # Release the VideoCapture
    cv2.destroyAllWindows()  # Close all OpenCV windows

print('Function split_camera_views created successfully')



Input folder = C:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\2_MOTION_TRACKING\1_Video_Segmentation\videos_raw
Output folder = C:\Users\ahmar\OneDrive\Documents\GitHub\Mobile-Multimodal-Lab\2_PREPROCESSING\2_MOTION_TRACKING\1_Video_Segmentation\videos_split
Function split_camera_views created successfully


## 2. Identifying Relevant Videos in the input_folder 

In [4]:
video_files = [ ]    # Initialize an empty list to store paths of video files

# Traverse through the directory and its subdirectories to find XDF files
for root, dirs, videos in os.walk(input_folder):  # 1st loop iterating over the results returned by os.walk().
    
    for video in videos:                               # 2nd loop iterating through each file in the current directory
        
        if video.endswith(video_extension):            # checking if the video has the specified video_extension (e.g., '.avi') 
            
             video_files.append(os.path.join(root, video))   # if the video is an .avi file, append its full path to the  list
            
print('We have idenified the following vides: ' + str(video_files))

We have idenified the following vides: ['./videos_raw/calibration_oneover250s_high_quality_2024-04-23_output_compr.avi', './videos_raw/oneover250s_highquality_test_2024-04-23_output_compr.avi', './videos_raw/oneover60_lowquality_test_2024-04-23_output_compr.avi']


## 2a. Alternatively, the user can select their own Videos from any directory. 

In [9]:
root = tkinter.Tk()
root.attributes('-topmost',True)
root.iconify()

video_extension = '.avi'  #Video extension. Change as needed

video_files = filedialog.askopenfilename(title="Select one or multiple Videos in the format" + str(video_extension), filetypes=[("Video Files", str(video_extension))], multiple = 'True')

root.destroy()

# Convert the tuple returned by askopenfilenames() to a list
video_files = list(video_files)

print('You have selected the following Videos: ' + str(video_files))

You have selected the following Videos: ['C:/Users/ahmar/OneDrive/Documents/GitHub/Mobile-Multimodal-Lab/2_PREPROCESSING/2_MOTION_TRACKING/1_Video_Segmentation/videos_raw/calibration_oneover250s_high_quality_2024-04-23_output_compr.avi']


## 3. Cutting Videos (Loop)

In [11]:
# loop over files in folder and split them
for file in video_files:
    
    print("working on file: "+file)
    
    # Get the name of the file without the extension
    filename = os.path.splitext(os.path.basename(file))[0]
    
    # trialID = filename.split("_")[0]
    
    # # create an empty folder with name of the sessionIndex
    # os.makedirs(os.path.join(outputfolder, trialID))
    # # inside this folder, create empty folder 'raw-2d'
    # os.makedirs(os.path.join(outputfolder, trialID, 'raw-2d'))

    # # create output file names, and save the three videos into the new created folder raw-2d within the sessionIndex folder
    # output_files = [
    #     os.path.join(outputfolder, trialID, 'raw-2d', filename + '_cam1.avi'),
    #     os.path.join(outputfolder, trialID, 'raw-2d', filename + '_cam2.avi'),
    #     os.path.join(outputfolder, trialID, 'raw-2d', filename + '_cam3.avi')
    # ]

        # create output file names, and save the three videos into the new created folder raw-2d within the sessionIndex folder
    output_files = [
        os.path.join(output_folder, 'raw-2d', filename + '_cam1.avi'),
        os.path.join(output_folder, 'raw-2d', filename + '_cam2.avi'),
        os.path.join(output_folder, 'raw-2d', filename + '_cam3.avi')
    ]



    # Split the camera views
    split_camera_views(file, output_files)

working on file: C:/Users/ahmar/OneDrive/Documents/GitHub/Mobile-Multimodal-Lab/2_PREPROCESSING/2_MOTION_TRACKING/1_Video_Segmentation/videos_raw/calibration_oneover250s_high_quality_2024-04-23_output_compr.avi
